# Imports

In [ ]:
%load_ext lab_black

import h5py
import os

from dataclasses import dataclass
from tqdm.auto import tqdm
from scipy.signal import savgol_filter
from scipy.interpolate import interp2d
from functools import lru_cache
import lmfit as lm

from typing import Dict, List, Optional, Tuple
import numpy as np
import plotly.graph_objects as go
import plotly.colors as pc
import matplotlib.pyplot as plt


import sys

sys.path.append(r"C:\Users\atully\Code\GitHub\ARPES Code\arpes-code-python")
from arpes_functions import (
    fitting_functions,
    analysis_functions,
    plotting_functions,
    HDF5_loader,
    misc_functions,
    filter_functions,
    tr_functions,
    loading_functions,
    cnn,
)

angstrom = "\u212B"

# Load Data

In [ ]:
ddir = r"E:\atully\arpes_data\2023_February\6eV\TR"
files = ["TR11_Ali_avg.h5"]  # 1.7 eV center energy; -1 to 100 ps

ARPES_DATA: Dict[str, tr_functions.ArpesData] = {}
ARPES_ATTRS: Dict[str, tr_functions.ArpesAttrs] = {}
for file in tqdm(files):
    data, theta, phi_or_time, energy = loading_functions.load_hdf5(ddir, file)
    ARPES_DATA[file] = tr_functions.ArpesData(
        data=data, theta=theta, phi_or_time=phi_or_time, energy=energy
    )
    ARPES_ATTRS[file] = tr_functions.load_attrs_hdf5(ddir, file)

ad_11 = ARPES_DATA[files[0]]

In [ ]:
ddir = r"E:\atully\arpes_data\2023_February\6eV\TR"
files = ["TR3_Ali_avg.h5"]  # 2.15 eV center energy; -1 to 2 ps

ARPES_DATA: Dict[str, tr_functions.ArpesData] = {}
ARPES_ATTRS: Dict[str, tr_functions.ArpesAttrs] = {}
for file in tqdm(files):
    data, theta, phi_or_time, energy = loading_functions.load_hdf5(ddir, file)
    ARPES_DATA[file] = tr_functions.ArpesData(
        data=data, theta=theta, phi_or_time=phi_or_time, energy=energy
    )
    ARPES_ATTRS[file] = tr_functions.load_attrs_hdf5(ddir, file)

ad_3 = ARPES_DATA[files[0]]

In [ ]:
ddir = r"E:\atully\arpes_data\2023_February\6eV\TR"
files = [
    "TR4_Ali_avg.h5"
]  # 2.6 eV center energy; -1 to 1 ps, same number of steps as first 2 ps of TR3

ARPES_DATA: Dict[str, tr_functions.ArpesData] = {}
ARPES_ATTRS: Dict[str, tr_functions.ArpesAttrs] = {}
for file in tqdm(files):
    data, theta, phi_or_time, energy = loading_functions.load_hdf5(ddir, file)
    ARPES_DATA[file] = tr_functions.ArpesData(
        data=data, theta=theta, phi_or_time=phi_or_time, energy=energy
    )
    ARPES_ATTRS[file] = tr_functions.load_attrs_hdf5(ddir, file)

ad_4 = ARPES_DATA[files[0]]

In [ ]:
ad = ad_4

for k in ["energy", "theta", "phi_or_time"]:
    print(f"{k}.shape = {getattr(ad, k).shape}")
print(f"Data.shape = {ad.data.shape}")

In [ ]:
print(f"Delay range (mm): {np.min(ad.phi_or_time), np.max(ad.phi_or_time)}")
print(
    f"Energy range (eV): {np.round(np.min(ad.energy), 2), np.round(np.max(ad.energy), 2)}"
)
print(f"Theta range: {np.round(np.min(ad.theta), 1), np.round(np.max(ad.theta), 1)}")

# Analysis

## Plan
ad_11 --> 1.7 eV center energy; -1 to 100 ps; 80 steps in delay (mm), but -1 to 1 ps in 21 steps (not great time resolution...)

ad_3 --> 2.15 eV center energy; -1 to 2 ps; 62 steps in delay (mm)

ad_4 --> 2.6 eV center energy; -1 to 1 ps; 42 steps in delay (mm)

1. cut down to same timescale (-1 to 1 ps --> 37.81 to 38.11 mm)
2. ensure appropriate x axis

In [ ]:
## Zero Delay ##

time_zero = 37.958  # from BiSe

## HOMO is at 2.05 eV below EF, based on fits from this data averaged with fits from tr-ARPES results ##

EF_400 = 1.91  # in kinetic energy, slit 400
EF_700 = 1.94  # in kinetic energy, slit 700

homo = -2.05

homo_400 = homo + EF_400
homo_700 = homo + EF_700

In [ ]:
## Integrate over desired angular range ##

slice_dim = "x"
slice_val = 0
int_range = 20  # if this value is more that the integration range, my get_2D_slice function will just integrate over the max range.

xlim = None
ylim = None
x_bin = 1
y_bin = 1

In [ ]:
all_vals = []
for ad in [ad_11, ad_3, ad_4]:
    all_vals.append(
        tr_functions.slice_datacube(
            ad_dataclass=ad,
            slice_dim=slice_dim,
            slice_val=slice_val,
            int_range=int_range,
            xlim=xlim,
            ylim=(
                ad.energy[57],
                ad.energy[1007],
            ),  # get rid of zero padding on datasets
            x_bin=x_bin,
            y_bin=y_bin,
            norm_data=False,
            plot_data=False,
        )
    )
x_11, y_11, d_11 = all_vals[0]
x_3, y_3, d_3 = all_vals[1]
x_4, y_4, d_4 = all_vals[2]

In [ ]:
## Plot Data: MPL ##

fig, ax = plt.subplots(1)

ax.pcolormesh(x_11, y_11, d_11, shading="auto", cmap="plasma", vmin=0, vmax=2.5)
ax.pcolormesh(x_3, y_3, d_3, shading="auto", cmap="plasma", vmin=0, vmax=0.2)
ax.pcolormesh(x_4, y_4, d_4, shading="auto", cmap="plasma", vmin=0, vmax=0.1)

ax.set_xlim(xmin=37.81, xmax=38.11)

# plt.save_fig(r'C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\TR3&TR4&T11_plasma_mpl.png')

In [ ]:
# # ## give data equivalent x axes ##

# # xlim = (37.81, 38.11)

# # x4, y4, d4 = analysis_functions.limit_dataset(x_4, y_4, d4_norm, xlim=xlim, ylim=None)
# # x3, y3, d3 = analysis_functions.limit_dataset(x_3, y_3, d_3, xlim=xlim, ylim=None)
# # x11, y11, d11 = analysis_functions.limit_dataset(
# #     x_11, y_11, d11_norm, xlim=xlim, ylim=None
# # )

# x4, y4, d4 = x_4, y_4, d4_norm
# x3, y3, d3 = x_3, y_3, d_3
# x11, y11, d11 = x_11, y_11, d11_norm

In [ ]:
# ## Linearly interpolate x11 to match resolution of TR3 and TR4 ##

# x, y, d = x11, y11, d11

# new_d = tr_functions.interpolate_dataset(x, y, d, xref=x3)

# fig = tr_functions.default_fig()
# fig.add_trace(go.Heatmap(x=x3, y=y, z=new_d))
# fig.show()

# print(new_d.shape)

In [ ]:
## Stitch Data ##

## TR4 & TR3
x_s1, y_s1, data_s1 = tr_functions.stitch_and_avg(
    x_4, y_4, d_4, x_3, y_3, d_3, no_avg=True
)

## TR4 & TR3 & TR11
x_s2, y_s2, data_s2 = tr_functions.stitch_and_avg(
    x_11, y_11, d_11, x_s1, y_s1, data_s1, no_avg=True
)

## TR11 & TR3
x_s3, y_s3, data_s3 = tr_functions.stitch_and_avg(
    x_11, y_11, d_11, x_3, y_3, d_3, no_avg=True
)

In [ ]:
## Plot Stitched Data ##

In [ ]:
yaxis_title = "E - E<sub>HOMO</sub> [eV]"
xaxis_title = "Delay [ps]"

In [ ]:
## Plot TR3 & TR4 ##
x_plot, y_plot, z_plot = x_s1, y_s1, data_s1

## toggle_time ?
toggle_time = "picoseconds"
# toggle_time = "mm"

## Logplot?
logplot = False
# logplot = True

if logplot:
    z_plot = np.log(z_plot)
    title = f"TR3 & TR4 (logplot)"
else:
    title = f"TR3 & TR4"

## Convert mm to ps
if toggle_time == "picoseconds":
    x_plot = tr_functions.mm_to_ps(x_plot, time_zero)

## Plot Data
fig = tr_functions.thesis_fig(
    title=title,
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    equiv_axes=False,
)
fig.add_trace(
    go.Heatmap(x=x_plot, y=y_plot, z=z_plot, coloraxis="coloraxis")
    # np.log(data)
)

fig.update_coloraxes(cmin=0, cmax=0.75)
fig.update_layout(width=800, height=600)
fig.show()

# fig.write_image(r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\TR3&TR4.png")

In [ ]:
## Plot TR11 & TR3 & TR4 ##
x_plot, y_plot, z_plot = x_s2, y_s2, data_s2

## toggle_time ?
toggle_time = "picoseconds"
# toggle_time = "mm"

## Logplot?
logplot = False
# logplot = True

if logplot:
    z_plot = np.log(z_plot)
    title = f"TR11 & TR3 & TR4 (logplot)"
else:
    title = f"TR11 & TR3 & TR4"

## Convert mm to ps
if toggle_time == "picoseconds":
    x_plot = tr_functions.mm_to_ps(x_plot, time_zero)

## Plot Data
fig = tr_functions.thesis_fig(
    title=title,
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    equiv_axes=False,
)
fig.add_trace(
    go.Heatmap(x=x_plot, y=y_plot, z=z_plot, coloraxis="coloraxis")
    # np.log(data)
)

fig.update_coloraxes(cmin=0, cmax=None)
fig.update_layout(width=800, height=600)
fig.show()

# fig.write_image(r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\TR11&TR3&TR4.png")

In [ ]:
## Plot TR11 & TR3 ##
x_plot, y_plot, z_plot = x_s3, y_s3, data_s3

## toggle_time ?
toggle_time = "picoseconds"
# toggle_time = "mm"

## Logplot?
logplot = False
# logplot = True

if logplot:
    z_plot = np.log(z_plot)
    title = f"TR11 & TR3 (logplot)"
else:
    title = f"TR11 & TR3"

## Convert mm to ps
if toggle_time == "picoseconds":
    x_plot = tr_functions.mm_to_ps(x_plot, time_zero)

## Plot Data
fig = tr_functions.thesis_fig(
    title=title,
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    equiv_axes=False,
)
fig.add_trace(go.Heatmap(x=x_plot, y=y_plot, z=z_plot, coloraxis="coloraxis"))

fig.update_coloraxes(cmin=0, cmax=None)
fig.update_layout(width=800, height=600)
fig.show()

# fig.write_image(r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\TR11&TR3.png")

# DiffMaps

In [ ]:
# Difference Map
title = "Difference Map of All States"

# d_diff = d_2d - d_2d[:, 2][:, None]
d_diff = data_s2 - np.mean(data_s2[:, 0:4], axis=1)[:, None]

# Plot Data
fig = tr_functions.thesis_fig(
    title=f"{title}",
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    equiv_axes=False,
    gridlines=False,
)

x_plot = tr_functions.mm_to_ps(x_s2, time_zero)

fig.add_trace(go.Heatmap(x=x_plot, y=y_s2, z=d_diff, coloraxis="coloraxis"))
fig.update_coloraxes(colorscale="RdBu", cmid=0, showscale=True)
# fig.update_layout(xaxis_range=[np.min(x_plot), 5])

In [ ]:
## Difference Map  TR11##
title = "Difference Map of TR11"
x, y, d = x_11, y_11, d_11

# d_diff = d_2d - d_2d[:, 2][:, None]
d_diff_11 = d - np.mean(d[:, 0:4], axis=1)[:, None]

# Plot Data
fig = tr_functions.thesis_fig(
    title=f"{title}",
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    equiv_axes=False,
    gridlines=False,
    height=400,
)

x_plot = tr_functions.mm_to_ps(x, time_zero)
y_plot = y

fig.add_trace(go.Heatmap(x=x_plot, y=y_plot, z=d_diff_11, coloraxis="coloraxis"))
# for h in [1.63, 1.8, 1.98]:
#     fig.add_hline(y=h, line=dict(color="black", width=1, dash="dash"))
fig.update_coloraxes(colorscale="RdBu", cmid=0, showscale=True)

In [ ]:
def set_up_EDC_fig(xaxis, title, fig=None):
    x_plot = xaxis
    if np.min(x_plot) > 0:
        x_edc = tr_functions.mm_to_ps(x_plot, time_zero)

    if logplot:
        title = f"{title} (logplot)"
    else:
        title = f"{title}"

    # Get and plot data
    if fig is None:
        fig = tr_functions.thesis_fig(
            title=title,
            xaxis_title="Energy (eV)",
            yaxis_title="Intensity (arb. u)",
            equiv_axes=False,
            gridlines=False,
            height=600,
            width=900,
        )

    return fig, x_edc

In [ ]:
def plot_EDC(
    x,
    y,
    data,
    fig=None,
    title="Title",
    logplot=False,
    xlim=None,
    ylim=None,
    show_plot=True,
):
    x_plot, y_plot, z_plot = x, y, data

    fig, x_edc = set_up_EDC_fig(x_plot, title)

    y_1d, col = tr_functions.get_1d_x_slice(
        x=x_edc,
        y=y_plot,
        data=z_plot,
        ylims=ylim,
        x_range=xlim,
    )

    if logplot:
        col = np.log(col)

    fig.add_trace(
        go.Scatter(
            x=y_1d,
            y=col,
            name=f"{np.round(xlim[0], 2), np.round(xlim[1], 2)} ps",
            line=dict(color=colors[0]),
        )
    )

    if show_plot:
        fig.update_layout(showlegend=True)
        fig.show()

    return fig, y_1d, col

In [ ]:
## Define Fit using FD + 1 or 2 Gaussians (and an offset) ##
import lmfit as lm


## FD
def fermi_dirac(x, center, theta, amp):
    arg = (x - center) / (2 * theta)  # x=E, center=mu, theta = k_B * T
    return -amp / 2 * np.tanh(arg)


def fit_FD_and_peaks(
    xaxis,
    data,
    params=None,
    num_peaks=2,
    peak_type="gaussian",
    center_peak1=1.65,
    center_peak2=1.8,
    center_FD=1.8,
    offset_type="linear",
    plot_fit=False,
):
    ## Offset
    offset_type = offset_type
    x = xaxis

    c = np.mean(data)
    b = (data[-1] - data[0]) / (x[-1] - x[0])
    a = 0

    offset = fitting_functions.offset_model(offset_type, a, b, c)

    ## Gaussians
    if peak_type == "gaussian":
        gauss1 = fitting_functions.make_gaussian(
            num="A_", amplitude=1, center=center_peak1, sigma=0.5
        )
        gauss2 = fitting_functions.make_gaussian(
            num="B_", amplitude=1, center=center_peak2, sigma=0.5
        )

        ## Full model
        if num_peaks == 2:
            full_model = lm.models.Model(fermi_dirac) + gauss1 + gauss2 + offset
        elif num_peaks == 1:
            full_model = lm.models.Model(fermi_dirac) + gauss1 + offset

    ## Lorentzians
    elif peak_type == "lorentzian":
        lorentz1 = fitting_functions.make_lorentzian(
            num="A_", amplitude=1, center=1.65, sigma=0.5
        )
        lorentz2 = fitting_functions.make_lorentzian(
            num="B_", amplitude=1, center=1.8, sigma=0.5
        )

        ## Full model
        if num_peaks == 2:
            full_model = lm.models.Model(fermi_dirac) + lorentz1 + lorentz2 + offset
        elif num_peaks == 1:
            full_model = lm.models.Model(fermi_dirac) + lorentz1 + offset

    else:
        raise ValueError(f"peak_type must be gaussian or lorentzian, not {peak_type}.")

    if params and offset_type == "constant":
        params["b"].value = 0
        params["b"].vary = False

    elif params is None:
        params = full_model.make_params()

        params["center"].value = center_FD
        params["theta"].value = k_B * (10.6)
        # params["theta"].min = 0
        params["amp"].value = 1

    fit = full_model.fit(data, x=xaxis, params=params)
    if plot_fit:
        fit.plot()
    return fit

In [ ]:
def plot_EDC_with_fit(
    x,
    y,
    data,
    fit=None,
    fig=None,
    title="Title",
    logplot=False,
    xlim=None,
    ylim=None,
    show_plot=True,
):
    colors = pc.qualitative.D3

    x_plot, y_plot, z_plot = x, y, data

    # fig = plot_EDC(x, y, data, fig, title, logplot, xlim, ylim, show_plot=False)
    fig, x_edc = set_up_EDC_fig(x_plot, title)

    # if np.min(x_plot) > 0:
    #     x_edc = tr_functions.mm_to_ps(x_plot, time_zero)

    y_1d, col = tr_functions.get_1d_x_slice(
        x=x_edc,
        y=y_plot,
        data=z_plot,
        ylims=ylim,
        x_range=xlim,
    )

    if fit is None:
        fit = fit_FD_and_peaks(xaxis=y_1d, data=col)
        print(
            "You didn't input a fit! Data was fit with default parameters -- CHECK THEM!"
        )

    fig.add_trace(
        go.Scatter(
            x=y_1d,
            y=col,
            name=f"{np.round(xlim[0], 2), np.round(xlim[1], 2)} ps",
            line=dict(color=colors[0]),
        )
    )

    fig.add_trace(go.Scatter(x=y_1d, y=fit.eval(x=y_1d), name="fit"))

    fig.update_layout(showlegend=True)

    fig.show()

    print(f"Center FD = {fit.params['center'].value:.2f} eV")
    print(f"T = {fit.params['theta'].value / k_B:.2f} K")
    print(f"Center A = {fit.params['iA__center'].value:.2f} eV")
    print(f"FWHM A = {fit.params['iA__fwhm'].value:.3f} ps")
    print(f"Center B = {fit.params['iB__center'].value:.2f} eV")
    print(f"FWHM B = {fit.params['iB__fwhm'].value:.3f} ps")

    return fig, fit

In [ ]:
def plot_fit_components(fit, title, show_plot=True):
    fig = tr_functions.thesis_fig(
        title=f"{title}<br> Fit Components",
        xaxis_title="Energy (eV)",
        yaxis_title="Intensity (arb. u)",
        equiv_axes=False,
        gridlines=False,
        height=600,
        width=900,
    )

    components = fit.eval_components(x=y_1d)

    for model_name, model_value in components.items():
        fig.add_trace(
            go.Scatter(
                x=y_1d,
                y=model_value,
                name=model_name,
            )
        )

    fig.data[3].update(name="offset")

    if show_plot:
        fig.show()
    return fig

In [ ]:
## Difference Map  TR3##
title = "Difference Map of TR3"
x, y, d = x_3, y_3, d_3

d_diff_3 = d - np.mean(d[:, 0:4], axis=1)[:, None]

# Plot Data
fig = tr_functions.thesis_fig(
    title=f"{title}",
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    equiv_axes=False,
    gridlines=False,
)

x_plot = tr_functions.mm_to_ps(x, time_zero)
y_plot = y - homo

fig.add_trace(go.Heatmap(x=x_plot, y=y_plot, z=d_diff_3, coloraxis="coloraxis"))
for h in [1.99, 2.15, 2.52]:
    fig.add_hline(y=h, line=dict(color="black", width=3, dash="dash"))
fig.update_coloraxes(colorscale="RdBu", cmid=0, showscale=True)

In [ ]:
## Difference Map  TR4##
title = "Difference Map of TR4"
x, y, d = x_4, y_4, d_4

# d_diff = d_2d - d_2d[:, 2][:, None]
d_diff_4 = d - np.mean(d[:, 0:4], axis=1)[:, None]

# Plot Data
fig = tr_functions.thesis_fig(
    title=f"{title}",
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    equiv_axes=False,
    gridlines=False,
)

x_plot = tr_functions.mm_to_ps(x, time_zero)
y_plot = y - homo

fig.add_trace(go.Heatmap(x=x_plot, y=y_plot, z=d_diff_4, coloraxis="coloraxis"))
for h in [2.52, 2.75, 2.96]:
    fig.add_hline(y=h, line=dict(color="black", width=3, dash="dash"))
fig.update_coloraxes(colorscale="RdBu", cmid=0, showscale=True)